<a href="https://colab.research.google.com/github/harshnadar/Codefest_enigma/blob/main/Codefest_Enigma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd 'drive'

/content/drive


In [3]:
cd 'My Drive'

/content/drive/My Drive


In [4]:
cd 'enigma authors'

/content/drive/MyDrive/enigma authors


#IMPORTING LIBRARIES

We have used pandas for loading the data. Then we have used numpy for fast processing and at last keras neural models to fit the data.

In [5]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import keras
import tensorflow
from sklearn.metrics import mean_absolute_error,mean_squared_error,accuracy_score,f1_score,precision_score,recall_score,r2_score
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LeakyReLU,ReLU,Activation
from keras.layers import Embedding,Input, concatenate, Reshape
from keras.models import *
from keras import optimizers
from keras.layers.normalization import BatchNormalization
from collections import Counter

#Loading the data

In [6]:
df_train=pd.read_csv('train.csv')
df_train.set_index('id',inplace=True)
df_train.head(5)
df_test=pd.read_csv('test.csv')
df_test.shape
df_train.head(5)

,author_id,year,venue_id,citations
id,,,,
162253,"['302080', '254034', '1650574', '1688060', '35...",2011.0,15632,0
42448,['471182'],2010.0,15752,0
226093,['1252574'],2012.0,19098,0
281211,"['1700521', '1634966', '1560119', '1588537', '...",2012.0,20323,29
8973,"['750555', '1125222']",2010.0,12539,0


Calculated mean for year and venue. We first converted venue to index then calculated the mean of both

In [7]:
df_venue=df_train.pivot_table(index=['venue_id'],values=['citations'],aggfunc='mean',fill_value=0)
vtoc=df_venue.to_dict('id')


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,


In [8]:
df_year=df_train.pivot_table(index=['year'],values=['citations'],aggfunc='mean',fill_value=0)
vtoc1=df_year.to_dict('id')

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,


#Created feature of venue and year
We have put default values for year and venue both as 3(by observation). Then we have included those features in our data.

In [9]:
def c1(l):
  d1 = vtoc.get(int(l),0)
  if (d1==0):
    return 3
  else:
    return d1['citations']
def c2(l):
  d1 = vtoc1.get(int(l),0)
  if (d1==0):
    return 3
  else:
    return d1['citations']
df_train['count']=df_train['venue_id'].apply(c1)
df_test['count']=df_test['venue_id'].apply(c1)
df_train['county']=df_train['year'].apply(c2)
df_test['county']=df_test['year'].apply(c2)
df_train.head(5)
df_test.head(5)

,author_id,year,venue_id,id,count,county
0,['1138048'],2010.0,22698,77152,5.224719,5.042993
1,"['1111668', '1154294']",2012.0,1037,253904,16.000000,5.581043
2,"['1505756', '1382981']",2010.0,22438,46386,8.000000,5.042993
3,"['634857', '542921']",2010.0,21723,19572,15.166667,5.042993
4,"['1709244', '1240251', '490730', '1525100']",2012.0,22453,182480,7.100000,5.581043


In [10]:
df_author=pd.read_csv('authors.csv')
df_author.set_index('id',inplace=True)
df_author=df_author[['published-papers','citations','h-index','p-equal-a','p-unequal-a','interests']]

#FINDING THE TOPIC OF THE PAPER BY COMMON INTEREST OF THE AUTHORS
We have made the vocab and word to index for the interest section of the authors.

In [11]:
interests=df_author[['interests']].values
d={}
vocab=[]
word_to_index={}
index=0
for i in interests:
  l=i[0].split(",")
  l=[i.strip("[").strip("]").strip(" ").strip("'") for i in l]
  for j in l:
    if (d.get(j,0)==0):
      d[j]=1
      vocab.append(j)
      word_to_index[j]=index
      index=index+1
    else:
      d[j]=d[j]+1
len(vocab)

2494140

#Calculating more features by taking into account the authors of the paper

First we have calculated the no. of authors for each paper and then we have calculated the maximum of 'published papers', 'citations', 'h-index', 'p-equal-a', 'p-unequal-a' for all the authors of particular paper and take those four values as the new feature for each paper

In [12]:
def cp(l):
    l1=l.split(",")
    l2=[]
    for i in l1:
        str1=i.strip('[')
        str2=str1.strip(']')
        str2=str2.strip(' ')
        str2=str2.strip("'")
        num=int(str2)
        sum1=0
        sum1=max(sum1,df_author.loc[num]['published-papers'])
       
        sum2=0
        sum2=max(sum2,df_author.loc[num]['citations'])
        
        sum3=0
        sum3=max(sum3,df_author.loc[num]['h-index'])
        
        sum4=0
        sum4=max(sum4,df_author.loc[num]['p-equal-a'])
       
        sum5=0
        sum5=max(sum5,df_author.loc[num]['p-unequal-a'])
       
    return sum1,sum2,sum3,sum4,sum5

In [13]:
top1=np.zeros((df_train.shape[0],1))
top2=np.zeros((df_train.shape[0],1))
top3=np.zeros((df_train.shape[0],1))

In [14]:
top1_t=np.zeros((df_test.shape[0],1))
top2_t=np.zeros((df_test.shape[0],1))
top3_t=np.zeros((df_test.shape[0],1))

#CALCULATED TOP 3 TOPICS FOR EACH PAPER

As written above that I have calculated interest's vocab and word to index. Then for each author of the paper we calculated the max common 3 interests among the authors and used them as features. Word to index was used to calculate the index of each interest. 

In [15]:
for i in range(len(df_train)):
  authors=df_train.iloc[i]['author_id']
  authors=[i.strip('[').strip(']').strip(' ').strip("'") for i in authors.split(",")]
  interest_list=[]
  for j in authors:
    interests=df_author.loc[int(j)]['interests']
    interests=[i.strip('[').strip(']').strip(' ').strip("'") for i in interests.split(",")]
    interest_list.extend(interests)
  dict1=Counter(interest_list)
  index1=1
  for k in dict1:
    k=str(k)
    if (index1==1):
      top1[i][0]=word_to_index[k]
    if (index1==2):
      top2[i][0]=word_to_index[k]
    if (index1==3):
      top3[i][0]=word_to_index[k]
    index1=index1+1
    if (index1==4):
      break

#Similarly did for test data

In [16]:
for i in range(len(df_test)):
  authors=df_test.iloc[i]['author_id']
  authors=[i.strip('[').strip(']').strip(' ').strip("'") for i in authors.split(",")]
  interest_list=[]
  for j in authors:
    interests=df_author.loc[int(j)]['interests']
    interests=[i.strip('[').strip(']').strip(' ').strip("'") for i in interests.split(",")]
    interest_list.extend(interests)
  dict1=Counter(interest_list)
  index1=1
  for k in dict1:
    k=str(k)
    if (index1==1):
      top1_t[i][0]=word_to_index[k]
    if (index1==2):
      top2_t[i][0]=word_to_index[k]
    if (index1==3):
      top3_t[i][0]=word_to_index[k]
    index1=index1+1
    if (index1==4):
      break

#Concatenating the features for train and test data

In [17]:
df_train['pp'], df_train['c'], df_train['hi'], df_train['p4'], df_train['p5'] = zip(*df_train['author_id'].map(cp))

In [18]:
df_test['pp'], df_test['c'], df_test['hi'], df_test['p4'], df_test['p5'] = zip(*df_test['author_id'].map(cp))

In [19]:
f1=df_test['id'].values
f1.shape

(148144,)

#Dividing into X and y

In [22]:
X=df_train[['pp','c','count','county']].values
X_t=df_test[['pp','c','count','county']].values
y=df_train[['citations']].values

#Normalising the data

In [23]:
scalar=StandardScaler()

In [24]:
X=scalar.fit_transform(X)
X

array([[-0.31633253, -0.20304571, -0.49423404, -0.4824248 ],
       [-0.39074924, -0.20602529, -0.84366104, -0.72070043],
       [ 0.576668  , -0.06300571, -0.84366104,  0.05195316],
       ...,
       [-0.13029075, -0.1464338 ,  1.93564475, -0.72070043],
       [-0.39074924, -0.20602529, -0.65664115,  0.05195316],
       [-0.35354089, -0.20602529,  0.70544902, -0.4824248 ]])

In [26]:
X_t=scalar.transform(X_t)

#SPLITTING THE DATASET

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=2018)
top1_train, top1_test, _, __ = train_test_split(top1, y, test_size=0.1, random_state=2018)
top2_train, top2_test, _, __ = train_test_split(top1, y, test_size=0.1, random_state=2018)
top3_train, top3_test, _, __ = train_test_split(top1, y, test_size=0.1, random_state=2018)

#Model
we have used the embedding layer to convert the top 3 topics for each paper to embedded vector of 8 dimensions
Then we have applied a simple neural model with activaton function "selu".

In [30]:
one_i=Input(shape=(1,),name='top1_in')
one_o=(Embedding(len(vocab), 8, mask_zero=False))(one_i)
one_o=(Reshape(target_shape=(8,),input_shape=(1,8)))(one_o)

two_i=Input(shape=(1,),name='top2_in')
two_o=(Embedding(len(vocab), 8, mask_zero=False))(two_i)
two_o=(Reshape(target_shape=(8,),input_shape=(1,8)))(two_o)

th_i=Input(shape=(1,),name='top3_in')
th_o=(Embedding(len(vocab), 8, mask_zero=False))(th_i)
th_o=(Reshape(target_shape=(8,),input_shape=(1,8)))(th_o)

main_i=Input(shape=(X_train.shape[1],),name='main_in')

x = concatenate([main_i, one_o, two_o, th_o])

x=(Dense(64,activation='selu'))(x)
x=(Dense(32,activation='selu'))(x)
x=(Dense(16,activation='selu'))(x)
x=(Dense(4,activation='selu'))(x)

x=(Dense(1,activation='selu'))(x)

model=Model(inputs=[one_i,two_i,th_i,main_i],outputs=x)
model.compile(optimizer='adam', loss='mean_squared_error')
print (model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
top1_in (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
top2_in (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
top3_in (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 1, 8)         19953120    top1_in[0][0]                    
______________________________________________________________________________________________

#Fitting the model


In [31]:
model.fit([top1_train,top2_train,top3_train,X_train],y_train,batch_size=1024, epochs=5, validation_split=0.3, verbose=1)

Epoch 1/5
137/137 [==============================] - 91s 544ms/step - loss: 60.6223 - val_loss: 34.6068
Epoch 2/5
137/137 [==============================] - 73s 536ms/step - loss: 33.3275 - val_loss: 34.1392
Epoch 3/5
137/137 [==============================] - 73s 534ms/step - loss: 26.2537 - val_loss: 35.0615
Epoch 4/5
137/137 [==============================] - 73s 533ms/step - loss: 22.7678 - val_loss: 35.8175
Epoch 5/5
137/137 [==============================] - 73s 534ms/step - loss: 21.1088 - val_loss: 36.7679


#Predicting the model

In [32]:
y_predict=model.predict([top1_t,top2_t,top3_t,X_t])
y_predict.shape

(148144, 1)

In [33]:
y_predict=y_predict.reshape(y_predict.shape[0],1)

In [34]:
f1=f1.reshape(f1.shape[0],1)

In [35]:
f2=np.hstack((f1,np.round(y_predict)))

In [36]:
f2.shape

(148144, 2)

In [37]:
answer=pd.DataFrame(f2,columns=['id','citations'])

In [38]:
answer.head(5)

,id,citations
0,77152.0,6.0
1,253904.0,18.0
2,46386.0,9.0
3,19572.0,15.0
4,182480.0,9.0


#saved the csv file :)))


In [39]:
answer.to_csv('submission(1).csv')